In [1]:
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
CSV_PATH = '../options_SPY_calls.csv'
df = pd.read_csv(CSV_PATH)
required_cols = {'S0','K','T','C_mkt'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f'Missing required columns: {missing}')
S0 = float(df['S0'].iloc[0])
df['T'] = (np.floor(df['T'] * 10) / 10).astype(float)
df = df.drop_duplicates(subset=['K','T']).reset_index(drop=True)
df = df[df['T'] > 0.0].reset_index(drop=True)
print(f'Loaded {CSV_PATH} with {len(df)} rows | Reference S0 = {S0:.4f}')
display(df.head())


Loaded ../options_SPY_calls.csv with 1289 rows | Reference S0 = 667.8400


,S0,K,C_mkt,T,type,iv
0,667.84,600.0,88.29,0.1,C,0.28
1,667.84,610.0,76.79,0.1,C,0.27
2,667.84,620.0,56.65,0.1,C,0.26
3,667.84,625.0,52.00,0.1,C,0.25
4,667.84,630.0,47.75,0.1,C,0.24


In [3]:
def normal_cdf(x: float) -> float:
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def bs_call_price(S0, K, T, r, q, vol):
    if T <= 0:
        return max(S0 * math.exp(-q * T) - K * math.exp(-r * T), 0.0)
    vol = max(vol, 1e-8)
    sqrt_T = math.sqrt(T)
    d1 = (math.log(S0 / K) + (r - q + 0.5 * vol * vol) * T) / (vol * sqrt_T)
    d2 = d1 - vol * sqrt_T
    discount_dom = math.exp(-r * T)
    discount_for = math.exp(-q * T)
    return S0 * discount_for * normal_cdf(d1) - K * discount_dom * normal_cdf(d2)

def implied_vol(price, S0, K, T, r=0.04, q=0.0):
    if T <= 0 or price <= 0:
        return float('nan')
    intrinsic = max(S0 * math.exp(-q * T) - K * math.exp(-r * T), 0.0)
    if price < intrinsic - 1e-12:
        return float('nan')
    sigma = 0.3
    sqrt_T = math.sqrt(T)
    for _ in range(100):
        model = bs_call_price(S0, K, T, r, q, sigma)
        diff = model - price
        if abs(diff) < 1e-6:
            return sigma
        d1 = (math.log(S0 / K) + (r - q + 0.5 * sigma * sigma) * T) / (max(sigma, 1e-8) * sqrt_T)
        vega = S0 * math.exp(-q * T) * sqrt_T * (1.0 / math.sqrt(2 * math.pi)) * math.exp(-0.5 * d1 * d1)
        if vega < 1e-10:
            break
        sigma -= diff / vega
        if sigma <= 0:
            sigma = 1e-4
    return sigma

df['iv_bs'] = df.apply(lambda row: implied_vol(row['C_mkt'], row['S0'], row['K'], row['T'], r=0.04), axis=1)
print('Computed Black-Scholes implied vols; sample:')
display(df[['K','T','iv_bs']].head())


Computed Black-Scholes implied vols; sample:


,K,T,iv_bs
0,600.0,0.1,0.560802
1,610.0,0.1,0.492614
2,620.0,0.1,0.288403
3,625.0,0.1,0.275465
4,630.0,0.1,0.268323


In [4]:
K_grid = np.arange(np.floor((S0 - 100.0)/10)*10, np.floor((S0 + 100.0)/10)*10 + 1, 1)
T_grid = np.arange(0.1, 2.0, 0.1)
KK, TT = np.meshgrid(K_grid, T_grid)

print (K_grid)
print (T_grid)

k_idx = np.abs(df['K'].to_numpy()[:, None] - K_grid).argmin(axis=1)
t_idx = np.abs(df['T'].to_numpy()[:, None] - T_grid).argmin(axis=1)
df['K_bin'] = K_grid[k_idx]
df['T_bin'] = T_grid[t_idx]
grouped = df.groupby(['T_bin','K_bin'], as_index=False)['iv_bs'].mean()
IV_surface = np.full((len(T_grid), len(K_grid)), np.nan)
k_pos = {v: idx for idx, v in enumerate(K_grid)}
t_pos = {v: idx for idx, v in enumerate(T_grid)}
for row in grouped.itertuples(index=False):
    IV_surface[t_pos[row.T_bin], k_pos[row.K_bin]] = row.iv_bs
for i in range(IV_surface.shape[0]):
    for j in range(IV_surface.shape[1]):
        if np.isnan(IV_surface[i, j]):
            left = IV_surface[i, j - 1] if j - 1 >= 0 else np.nan
            right = IV_surface[i, j + 1] if j + 1 < IV_surface.shape[1] else np.nan
            neighbors = [val for val in (left, right) if not np.isnan(val)]
            if neighbors:
                IV_surface[i, j] = np.mean(neighbors)
if np.isnan(IV_surface).any():
    IV_surface = np.where(np.isnan(IV_surface), np.nanmean(IV_surface), IV_surface)
print(f'Grid populated with Black-Scholes IVs; remaining NaNs filled by neighbor averaging.')


[560. 561. 562. 563. 564. 565. 566. 567. 568. 569. 570. 571. 572. 573.
 574. 575. 576. 577. 578. 579. 580. 581. 582. 583. 584. 585. 586. 587.
 588. 589. 590. 591. 592. 593. 594. 595. 596. 597. 598. 599. 600. 601.
 602. 603. 604. 605. 606. 607. 608. 609. 610. 611. 612. 613. 614. 615.
 616. 617. 618. 619. 620. 621. 622. 623. 624. 625. 626. 627. 628. 629.
 630. 631. 632. 633. 634. 635. 636. 637. 638. 639. 640. 641. 642. 643.
 644. 645. 646. 647. 648. 649. 650. 651. 652. 653. 654. 655. 656. 657.
 658. 659. 660. 661. 662. 663. 664. 665. 666. 667. 668. 669. 670. 671.
 672. 673. 674. 675. 676. 677. 678. 679. 680. 681. 682. 683. 684. 685.
 686. 687. 688. 689. 690. 691. 692. 693. 694. 695. 696. 697. 698. 699.
 700. 701. 702. 703. 704. 705. 706. 707. 708. 709. 710. 711. 712. 713.
 714. 715. 716. 717. 718. 719. 720. 721. 722. 723. 724. 725. 726. 727.
 728. 729. 730. 731. 732. 733. 734. 735. 736. 737. 738. 739. 740. 741.
 742. 743. 744. 745. 746. 747. 748. 749. 750. 751. 752. 753. 754. 755.
 756. 

In [5]:
fig = go.Figure(data=[go.Surface(x=KK, y=TT, z=IV_surface, colorscale='Viridis', opacity=0.85)])
fig.update_layout(
    title='Black-Scholes Implied Volatility Surface',
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Implied Volatility'
    ),
    template='plotly_dark',
    width=900,
    height=600
)
fig.show()
